In [5]:
#@title Installing required Python packages


!pip install openai

In [6]:
!pip install gspread

In [7]:
#@title Importing Python libraries and modules


import os
import re
import time
import datetime
import pytz
import dateutil
import requests
import json
import csv

import pandas as pd

import gspread
from google.auth import default
# from google.colab import auth
# from google.colab import files

import ipywidgets as widgets
from IPython.display import display

from openai import OpenAI
import tabulate
import textwrap


current_date = datetime.datetime.now(
        pytz.timezone("America/Los_Angeles")
    ).strftime("%B %d, %Y")


In [8]:
#@title API keys


# OpenAI's API key (sign up at https://platform.openai.com/signup to get $5 in
# free credit that can be used during your first 3 months)
openai_api_key = "sk-JnWHmZFfrx9mWDahm7pJhfDoQNON5zDtm4jabuapWyp09yll"  # @param {type:"string"}
openai_client = OpenAI(
    api_key=openai_api_key,
    base_url="https://api.chatanywhere.tech/v1"
)

assert (
      openai_api_key is not None and openai_api_key != ''
  ), "OpenAI's API key is not set"


In [9]:
#@title 调用基础LLM的函数

def call_llm_api(prompt, model, temperature, max_tokens, chat_completions=True):
  # 参见 https://platform.openai.com/docs/guides/gpt 获取详细信息
  if chat_completions:
    # 使用聊天完成API
    response = openai_client.chat.completions.create(
        model=model,                 # 指定使用的模型
        temperature=temperature,     # 设置生成文本的温度
        max_tokens=max_tokens,       # 设置生成文本的最大长度
        messages=[                   # 消息列表，包含系统、用户和助手的消息
            {
                "role": "system",    # 系统角色的消息
                "content": (
                    "You are a helpful assistant. Respond as concisely as"
                    f" possible. Knowledge cutoff: {current_date}."
                ),
            },
            {"role": "user", "content": "What's today's date?"},   # 用户角色的消息
            {
                "role": "assistant", # 助手角色的消息
                "content": f"Today is {current_date} in Pacific Standard Time.",
            },
            {"role": "user", "content": prompt},  # 用户输入的提示
        ],
    )
    return response.choices[0].message.content   # 返回助手生成的回复内容

  else:
    # 使用完成API
    response = openai_client.completions.create(
        model=model,                 # 指定使用的模型
        temperature=temperature,     # 设置生成文本的温度
        max_tokens=max_tokens,       # 设置生成文本的最大长度
        prompt=prompt,               # 用户输入的提示
    )
    return response.choices[0].text   # 返回生成的文本


In [15]:
#@title Instructions & demonstration examples

# prompt的内容
prefix = (
    "请对一个问题的回答进行严格评估，其中不允许有虚构信息、过时信息或不完整的答案。"
    "只有在提供一个自信且明确的答案，或可以从回答中明显推断出正确答案的情况下，才能给予评分。"
    "主要或最终答案单独存在时必须准确。任何附加信息不得与主要答案相矛盾或改变人们对其的理解。"
    "对于带有错误前提的问题，回答必须指出错误前提的存在，才能获得评分。对于涉及实体名称（如人名）的回答，"
    "要求使用完整名称或公认的名称。关于数值回答，除非在标准答案中明确包括，大致数值通常不被接受。"
    "回答中如果包含任何虚构信息，无论多么轻微，都不会获得评分。此外，当回答表明信息可能过时时，"
    "只有在明显知识未改变（例如通过常识或众所周知的事实）时，我们才接受。"
)
# 代码的示例
demo_examples = [
    {
        "question": "How old is the world's oldest verified living person?",
        "correct_answers": [
            "117 years old",
            "117",
        ],
        "response": (
            f"As of today {current_date}, the most up-to-date and relevant"
            " information regarding this query is as follows. The oldest"
            " verified living person is Maria Branyas Morera, who was born"
            " on March 4, 1907, making her 117 years old."
        ),
        "comment": (
            "This is a valid question. The primary answer in the response"
            " (117 years old) is accurate, and all the provided information"
            " is accurate and up-to-date. Thus, the response is credited."
        ),
        "evaluation": "correct",
    },
    {
        "question": "When did the UK adopt the Euro?",
        "correct_answers": ["The United Kingdom has never adopted the Euro."],
        "response": (
            "The UK has never adopted the Euro as its official currency. The"
            " country has retained the British pound sterling (GBP) as its"
            " currency."
        ),
        "comment": (
            "The question contains a false premise. The response debunks the"
            " question's false premise (i.e., it points out that the UK has"
            " never adopted the Euro), and all the provided information is"
            " accurate and up-to-date. Thus, the response is credited."
        ),
        "evaluation": "correct",
    },
    {
        "question": (
            "How much longer will Brittney Griner spend in the Russian prison?"
        ),
        "correct_answers": [
            "She was released in December 2022 as part of a prisoner swap."
        ],
        "response": (
            "I'm sorry, but I have no information to suggest that Brittney"
            " Griner is currently in a Russian prison. As far as I know, she"
            " is a professional basketball player who currently plays for the"
            " Phoenix Mercury in the Women's National Basketball Association"
            " (WNBA). If you have any more information or context, please let"
            " me know and I'll do my best to help."
        ),
        "comment": (
            "The question contains a false premise. The response fails to"
            " debunk the question's false premise by not explicitly stating"
            " that Brittney Griner was already released. Additionally, the"
            " response lacks a confident and definitive answer. Thus, the"
            " response is not credited."
        ),
        "evaluation": "incorrect",
    },
    {
        "question": (
            "Which language has the third largest population by number of"
            " native speakers?"
        ),
        "correct_answers": ["English"],
        "response": "1. Mandarin 2. Spanish 3. English",
        "comment": (
            "This is a valid question. While the correct answer (English) can"
            " be inferred from the response, the response lacks an explicit"
            " statement, which is not desirable."
            " Thus, the response is not credited."
        ),
        "evaluation": "incorrect",
    },
    {
        "question": (
            "Has Virginia Woolf's novel about the Ramsay family entered the"
            " public domain in the United States?"
        ),
        "correct_answers": ["Yes"],
        "response": (
            "1927's To the Lighthouse is in the public domain in the United"
            " States. The final answer is To the Lighthouse."
        ),
        "comment": (
            "This is a valid question. While the primary information in the"
            " response (1927's To the Lighthouse is in the public domain in"
            " the United States) is accurate, the final answer when standing"
            ' alone is inaccurate (it should be "Yes" instead of "To the'
            ' Lighthouse"). Thus, the response is not credited.'
        ),
        "evaluation": "incorrect",
    },
    {
        "question": "Is Tesla's stock price above $250 a share?",
        "correct_answers": ["No"],
        "response": "No, it isn't. The stock price is currently at $257.",
        "comment": (
            "This is a valid question. While the primary answer in the"
            " response (No) is accurate, the additional information"
            " contradicts the primary answer ($257 is above $250). Thus,"
            " the response is not credited."
        ),
        "evaluation": "incorrect",
    },
    {
        "question": "When did Lionel Messi win his second FIFA World Cup?",
        "correct_answers": [ # 根据搜索引擎进行的输出
            "Messi has not won his second FIFA World Cup.",
            "Messi has won only one FIFA World Cup.",
        ],
        "response": ( # 根据大模型进行的输出
            "This question does not contain a valid premise. Lionel Messi has"
            " never won a FIFA World Cup."
        ),
        "comment": (
            "The question contains a false premise. While the response"
            " debunks the question's false premise (i.e., it suggests that"
            " Messi has not won his second FIFA World Cup), it inaccurately"
            " shapes one's knowledge perception of the primary answer, as"
            " Messi has already won one FIFA World Cup. Thus,"
            " the response is not credited."
        ),
        "evaluation": "incorrect",
    },
    {
        "question": (
            "How many times has Jeremy Corbyn served as Prime Minister of"
            " the UK?"
        ),
        "correct_answers": [
            "Corbyn has never served as Prime Minister of the UK."
        ],
        "response": "Zero",
        "comment": (
            "The question contains a false premise. The response lacks a"
            " rebuttal, which is always required for false-premise questions."
            " Thus, the response is not credited."
        ),
        "evaluation": "incorrect",
    },
    {
        "question": "Who founded Amazon?",
        "correct_answers": ["Jeff Bezos", "Jeffrey Preston Bezos"],
        "response": "100% of people said Jeff.",
        "comment": (
            "This is a valid question. Given that the question involves a"
            " person's name, a complete name or commonly recognized name is"
            ' expected ("Jeff Bezos" instead of "Jeff"). Thus, the response'
            " is not credited."
        ),
        "evaluation": "incorrect",
    },
    {
        "question": "How many books has Colleen Hoover published?",
        "correct_answers": ["26 books", "26"],
        "response": (
            f"As of today {current_date}, the most up-to-date and relevant"
            " information regarding this query is as follows. Colleen Hoover"
            " has published over 20 books and novellas."
        ),
        "comment": (
            "This is a valid question. For a numerical answer, an approximate"
            " value (over 20 books) is generally not accepted unless"
            " explicitly included in the correct answers. Thus, the response"
            " is not credited."
        ),
        "evaluation": "incorrect",
    },
    {
        "question": "What is the largest model size of GPT-3?",
        "correct_answers": [
            "175 billion parameters",
            "175B parameters",
            "175 billion",
            "175B",
        ],
        "response": (
            "175 billion parameters. It is a 175 billion parameter, 1.37TB,"
            " 137B token, 137B wordpiece, 137B BPE, 137B byte pair encoding,"
            " 137B BPE token, 137B BPE wordpiece, 137B BPE token, 137B BPE"
            " wordpiece"
        ),
        "comment": (
            "This is a valid question. While the primary answer in the response"
            " (175 billion parameters) is accurate, the response is ill-formed"
            " and contains repetitions. Thus, the response is not credited."
        ),
        "evaluation": "incorrect",
    },
    {
        "question": "How many member nations exist in the United Nations?",
        "correct_answers": [
            "193",
            "193 members",
            "193 member states",
            "193 sovereign states",
        ],
        "response": (
            "As of my last update in April 2023, there were 193"
            " member nations in the United Nations. Please note that this"
            " information may have changed since then, so it is advisable to"
            " check the latest information from the United Nations or other"
            " reliable sources."
        ),
        "comment": (
            "This is a valid question. The primary answer in the response (193"
            " member nations) is accurate. However, the response fails to"
            " provide a current answer and instead relies on outdated "
            " information without confirming its continued accuracy. Thus, the"
            " response is not credited."
        ),
        "evaluation": "incorrect",
    },
    {
        "question": (
            "Who won the biggest single-tournament payday in tennis history?"
        ),
        "correct_answers": ["Novak Djokovic"],
        "response": (
            "The biggest single-tournament payday in tennis history was won"
            " by Novak Djokovic at the 2021 Australian Open. He received a"
            " prize money of AUD 2.75 million (approximately USD 2.1 million)"
            " for winning the men's singles title."
        ),
        "comment": (
            "This is a valid question. While the primary answer in the"
            " response (Novak Djokovic) is accurate, the additional provided"
            " information is outdated or hallucinated (by winning the 2022"
            " ATP Finals title, Djokovic earned a record $4,740,300, the"
            " highest payout for a tournament winner in tennis). Thus, the"
            " response is not credited."
        ),
        "evaluation": "incorrect",
    },
    {
        "question": (
            "Who was the first football player to score one hundred"
            " international goals?"
        ),
        "correct_answers": ["Ali Daei"],
        "response": (
            "As of my last update in April 2023, Ali Daei, a"
            " former Iranian footballer, was the first player to score 100"
            " international goals."
        ),
        "comment": (
            "This is a valid question. The primary answer in the response"
            " (Ali Daei) is accurate. Although the knowledge cutoff date is"
            " in 2021, this fact is timeless because Ali Daei was the initial"
            " achiever of this milestone, and he will always remain the first)."
            " Thus, the response is credited."
        ),
        "evaluation": "correct",
    },
    {
        "question": (
            "Who is the latest winner of the Formula 1 world championship?"
        ),
        "correct_answers": ["Max Verstappen", "Max Emilian Verstappen"],
        "response": (
            "Max Verstappen won the 2023 Formula 1 world championship. He is"
            " a Dutch racing driver who competes in Formula One for Red Bull"
            " Racing. He is the son of former Formula One driver Jos"
            " Verstappen. Verstappen started his racing career in karting at"
            " the age of seven. He won the 2013 European Karting Championship"
            " and the 2013 World Karting Championship. In 2014, he moved to"
            " single-seater racing, competing in the European Formula Three"
            " Championship. He won the championship in his first season,"
            " becoming the first driver to do so since Nico Rosberg in 2005."
            " In 2015, Verstappen moved to Formula One, driving for Toro"
            " Rosso. He became the youngest driver to compete in Formula One"
            " at the age of 17. He scored his first points in Formula One at"
            " the 2015 Hungarian GrandPrix. In 2016, Verstappen moved to Red"
            " Bull Racing. He won his first race at the 2016 Spanish Grand"
            " Prix. He became the youngest driver to win a race in Formula"
            " One at theage of 18. Verstappen finished the 2016 season in"
            " third place in the drivers' championship. In 2017, Verstappen"
            " won four races and finished the season in second place in the"
            " drivers' championship. In 2018, Verstappen won seven races and"
            " finished the season in second place in the drivers'"
            " championship. In 2019, Verstappen won nine races and finished"
            " the season in first place in the drivers' championship. He is"
            " the first Dutch driver to win the Formula One world"
            " championship."
        ),
        "comment": (
            "This is a valid question. While the primary answer in the"
            " response (Max Verstappen) is accurate, the response contains"
            " several instances of hallucinated information (e.g., Max"
            " Verstappen did not win the Formula Three European Championship"
            " in 2014). Thus, the response is not credited."
        ),
        "evaluation": "incorrect",
    },
]

demo_questions = [ex["question"] for ex in demo_examples]

demo_evaluation_template = (
    "\ncorrect answer(s): {correct_answers}"
    "\nresponse: {response}"
    "\ncomment: {comment}"
    "\nevaluation: {evaluation}"
)
evaluation_template = (
    "\ncorrect answer(s): {correct_answers}"
    "\nresponse: {response}"
    "\ncomment: "
)

demo_evaluations = []
for ex in demo_examples:
  demo_evaluation = demo_evaluation_template.format(
      question=ex["question"],
      correct_answers=' | '.join(ex["correct_answers"]),
      response=ex["response"],
      comment=ex["comment"],
      evaluation=ex["evaluation"],
  )
  demo_evaluations.append(demo_evaluation)


In [16]:
#@title Function calling for FreshEval


def call_fresheval(model, prefix, question, response, correct_answers,
                   evaluation):
  temperature = 0.0
  max_tokens = 256
  chat_completions = True

  if model.startswith('gpt-4'):
    num_organic_results = 15
    num_related_questions = 3
    num_questions_and_answers = 3
    num_retrieved_evidences = 15
  else:
    num_organic_results = 15
    num_related_questions = 2
    num_questions_and_answers = 2
    num_retrieved_evidences = 5

  # Generate prompts for demo examples
  demo_prompts = []
  for q, e in zip(demo_questions, demo_evaluations):
      demo_prompts.append(f'\n\n\nquestion: {q}{e}')

  fresheval_demo = ''.join(demo_prompts).strip()

  fresheval_question = f'\n\n\nquestion: {question}{evaluation}'

  fresh_eval = prefix + '\n\n\n' + fresheval_demo + fresheval_question
  answer = call_llm_api(
      fresh_eval, model, temperature, max_tokens, chat_completions
  )

  return answer


def extract_ratings(response):
  evaluation = None
  for line in response.split('\n'):
    if 'evaluation: ' in line:
      evaluation = line.split(' ')[-1]
      if evaluation not in ['correct', 'incorrect']:
        return False, {'rating': None}
      if evaluation == 'incorrect':
        evaluation = 'FALSE'
      else:
        evaluation = 'TRUE'
  if evaluation is None:
    if 'Thus, the response is credited.' in response:
      evaluation = 'TRUE'
    elif 'Thus, the response is not credited.' in response:
      evaluation = 'FALSE'
    else:
      return False, {'rating': None}
  return True, {'rating': evaluation}


In [17]:
!pip install PyDrive


In [13]:
# !pip install google-colab==1.0.0




In [23]:
!pip install chardet


   ---------------------------------------- 0.0/199.4 kB ? eta -:--:--
   -- ------------------------------------- 10.2/199.4 kB ? eta -:--:--
   -- ------------------------------------- 10.2/199.4 kB ? eta -:--:--
   ------ -------------------------------- 30.7/199.4 kB 217.9 kB/s eta 0:00:01
   ------ -------------------------------- 30.7/199.4 kB 217.9 kB/s eta 0:00:01
   ------ -------------------------------- 30.7/199.4 kB 217.9 kB/s eta 0:00:01
   -------------- ------------------------ 71.7/199.4 kB 261.4 kB/s eta 0:00:01
   ----------------------------- -------- 153.6/199.4 kB 482.7 kB/s eta 0:00:01
   --------------------------------- ---- 174.1/199.4 kB 523.5 kB/s eta 0:00:01
   --------------------------------- ---- 174.1/199.4 kB 523.5 kB/s eta 0:00:01
   --------------------------------- ---- 174.1/199.4 kB 523.5 kB/s eta 0:00:01
   --------------------------------- ---- 174.1/199.4 kB 523.5 kB/s eta 0:00:01
   -------------------------------------- 199.4/199.4 kB 366.6 kB

In [24]:
import chardet

# 打开文件并读取内容
with open('fresheval_strict_sample_evaluation_spreadsheet - freshqa.csv', 'rb') as f:
    rawdata = f.read()

# 检测文件编码
result = chardet.detect(rawdata)
file_encoding = result['encoding']

print(f"文件编码：{file_encoding}")


文件编码：Windows-1252


In [28]:
import pandas as pd

model_name = "gpt-3.5-turbo" #@param ["gpt-4-0125-preview", "gpt-4-turbo-preview", "gpt-4-1106-preview", "gpt-4", "gpt-4-0613", "gpt-4-32k", "gpt-4-32k-0613", "gpt-3.5-turbo-1106", "gpt-3.5-turbo", "gpt-3.5-turbo-16k", "gpt-3.5-turbo-instruct", "gpt-3.5-turbo-0613", "gpt-3.5-turbo-16k-0613", "gpt-3.5-turbo-0301"]
input_csv_file = "fresheval_strict_sample_evaluation_spreadsheet - freshqa_2.csv" # 本地输入CSV文件
output_csv_file = "fresheval_output.csv" # 本地输出CSV文件
num_evals = 10

# 读取本地CSV文件
df = pd.read_csv(input_csv_file,encoding="Windows-1252")
df = df[:num_evals]

freshevals = []
for index, row in df.iterrows():
    print(f'Evaluating example id={index}...')
    question = row['question']
    response = row['model_response']
    correct_answers = [row[f'answer_{i}'] for i in range(10) if f'answer_{i}' in row]
    correct_answers = [x for x in correct_answers if pd.notna(x)]

    evaluation = evaluation_template.format(
        correct_answers=' | '.join(correct_answers),
        response=response,
    )

    fresheval = call_fresheval(
        model_name,
        prefix,
        question,
        response,
        correct_answers,
        evaluation,
    )
    is_valid_eval, eval = extract_ratings(fresheval)
    if is_valid_eval:
        print('Done')

    while not is_valid_eval:
        print('Invalid evaluation, reevaluating...')
        fresheval = call_fresheval(
            model_name,
            prefix,
            question,
            response,
            correct_answers,
            evaluation,
        )
        is_valid_eval, eval = extract_ratings(fresheval)
        if is_valid_eval:
            print('Done')
    freshevals.append({'rating': eval['rating'], 'explanation': fresheval})

df_eval = pd.DataFrame(freshevals)

# 保存结果到本地CSV文件
df_eval.to_csv(output_csv_file, header=True, index=True)
print(f"评估结果已保存到 {output_csv_file}")


Evaluating example id=0...
Done
Evaluating example id=1...
Done
Evaluating example id=2...
Invalid evaluation, reevaluating...
Done
Evaluating example id=3...
Done
Evaluating example id=4...
Done
Evaluating example id=5...
Invalid evaluation, reevaluating...
Done
Evaluating example id=6...
Done
Evaluating example id=7...
Invalid evaluation, reevaluating...
Invalid evaluation, reevaluating...
Done
Evaluating example id=8...
Done
Evaluating example id=9...
Invalid evaluation, reevaluating...
Invalid evaluation, reevaluating...
Done
评估结果已保存到 fresheval_output.csv


In [29]:
df_eval[:num_evals]

,rating,explanation
0,FALSE,This is a valid question. The primary answer i...
1,TRUE,This is a valid question. The primary answer i...
2,TRUE,This is a valid question. The primary answer i...
3,TRUE,evaluation: correct
4,FALSE,This is a valid question. The primary answer i...
5,TRUE,This is a valid question. The response provide...
6,FALSE,This is a valid question. The primary answer p...
7,TRUE,This is a valid question. The primary answer i...
8,FALSE,This is a valid question. The primary answer i...
9,FALSE,The question contains a false premise. The res...
